<a href="https://colab.research.google.com/github/angelaoryza/BigData/blob/main/MidTerm_Exam/spark_stream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark Streaming

Name : Angela Oryza Prabowo

Student Number : 5025201022

# 1. Preparing the Environment

We are going to set up our environtment by defining the directory for loading the input files and storing the output files

In [25]:
import os.path
from os import path

if path.exists('/content/input-5025201022') == False:
  os.mkdir('/content/input-5025201022')


!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1B-ywFVoUqW9_qHaOkgGG8Q4qirwSWbo0' -O '/content/input-5025201022/news.zip'
!unzip '/content/input-5025201022/news' -d '/content/input-5025201022/'

--2023-04-14 02:44:55--  https://docs.google.com/uc?export=download&id=1B-ywFVoUqW9_qHaOkgGG8Q4qirwSWbo0
Resolving docs.google.com (docs.google.com)... 142.251.163.102, 142.251.163.139, 142.251.163.101, ...
Connecting to docs.google.com (docs.google.com)|142.251.163.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-40-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mf3a0nuo2am9rkeonhls2aigut8f00rk/1681440225000/08107738916201805538/*/1B-ywFVoUqW9_qHaOkgGG8Q4qirwSWbo0?e=download&uuid=78408170-d888-4a9e-b204-3795c3ce70a2 [following]
--2023-04-14 02:44:56--  https://doc-10-40-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mf3a0nuo2am9rkeonhls2aigut8f00rk/1681440225000/08107738916201805538/*/1B-ywFVoUqW9_qHaOkgGG8Q4qirwSWbo0?e=download&uuid=78408170-d888-4a9e-b204-3795c3ce70a2
Resolving doc-10-40-docs.googleusercontent.com (doc-10-40-docs.googleusercontent.com)... 172.253.62.132, 2607:f

#2. Initializing Apache Spark

In [26]:
# Install Apache Spark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Streaming Process Files") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .master("local[*]") \
    .getOrCreate()

spark

#3. Configuring the Apache Spark Streaming Machine

In [28]:
# To allow automatic schemaInference while reading
spark.conf.set("spark.sql.streaming.schemaInference", True)

# Create the streaming_df to read from input directory
streaming_df = spark.readStream\
    .format("json") \
    .option("cleanSource", "archive") \
    .option("sourceArchiveDir", "archive/") \
    .option("maxFilesPerTrigger", 1) \
    .load("/content/input-5025201022/news/news/")

In [29]:
# To the schema of the data, place a sample json file and change readStream to read 
streaming_df.printSchema()

root
 |-- authors: string (nullable = true)
 |-- category: string (nullable = true)
 |-- date: string (nullable = true)
 |-- headline: string (nullable = true)
 |-- link: string (nullable = true)
 |-- short_description: string (nullable = true)



# 4. Write the output into designated folders

In [30]:
# Write the output to console sink to check the output
writing_df = streaming_df.writeStream \
    .format("json") \
    .option("path", "output-5025201022/") \
    .option("checkpointLocation","checkpoint_dir") \
    .outputMode("append") \
    .start()
    
# Start the streaming application to run until the following happens
# 1. Exception in the running program
# 2. Manual Interruption
writing_df.awaitTermination(120)

False

We will stop the streaming after it has taken 120 seconds

In [31]:
# Check the data at the output location

out_df = spark.read.json("output-5025201022/")
out_df.show()

+--------------------+--------------+----------+--------------------+--------------------+--------------------+
|             authors|      category|      date|            headline|                link|   short_description|
+--------------------+--------------+----------+--------------------+--------------------+--------------------+
|                    |     U.S. NEWS|2022-09-04|Emergency Declare...|https://www.huffp...|Thunderstorms and...|
|   Marco Margaritoff| ENTERTAINMENT|2022-09-03|'The Osbournes' W...|https://www.huffp...|Millennials might...|
|                    |     U.S. NEWS|2022-09-03|Plane That Circle...|https://www.huffp...|A pilot has been ...|
|     Marcia Dunn, AP|       SCIENCE|2022-09-03|Fuel Leak Ruins N...|https://www.huffp...|Part of the space...|
|   Terry Wallace, AP|     U.S. NEWS|2022-09-03|9 Migrants Are Fo...|https://www.huffp...|Tragic toll follo...|
|        Ben Blanchet|     U.S. NEWS|2022-09-02|Owner Of New Engl...|https://www.huffp...|Nashua police 